In [1]:
import os
from config import *
from tqdm import tqdm
from utils.dataset import DrawingsDataset

import torch
import torch.nn.functional as F

from model import Net

In [2]:
train_data = DrawingsDataset(mtype="train")
train_loader = torch.utils.data.DataLoader(train_data, batch_size=MODEL_CFG['batch_size'], shuffle=True)

test_data = DrawingsDataset(mtype="test")
test_loader = torch.utils.data.DataLoader(test_data, batch_size=MODEL_CFG['batch_size'], shuffle=True)

print("Train images: %d" % len(train_data))
print("Test images: %d" % len(test_data))

net = Net().cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=MODEL_CFG["learning_rate"], momentum=MODEL_CFG['momentum'],
                            weight_decay=MODEL_CFG['weight_decay'])

train_accuracy = 0.0
test_accuracy = 0.0
best_accuracy = 0.0
    
def train():
    net.train()
    loss_avg = 0.0
    correct = 0
    
    data_loader = tqdm(train_loader, desc='Training')
    for batch_idx, (drawings, labels) in enumerate(data_loader):
        drawings, labels = torch.autograd.Variable(drawings.cuda()), torch.autograd.Variable(labels.cuda())
        drawings = drawings.view(-1, 1, IMAGE_SIZE, IMAGE_SIZE)
        drawings /= 255.0

        # forward
        output = net(drawings)

        # backward
        optimizer.zero_grad()
        loss = F.cross_entropy(output, labels)
        loss.backward()
        optimizer.step()

        # accuracy
        pred = output.data.max(1)[1]
        correct += float(pred.eq(labels.data).sum())

        # exp moving average
        loss_avg = loss_avg*0.2+float(loss)*0.8

    global train_accuracy
    train_accuracy = correct/len(train_loader.dataset)

def test():
    net.eval()
    loss_avg = 0.0
    correct = 0
    
    data_loader = tqdm(test_loader, desc='Testing')
    for batch_idx, (drawings, labels) in enumerate(data_loader):
        drawings, labels = torch.autograd.Variable(drawings.cuda()), torch.autograd.Variable(labels.cuda())
        
        drawings = drawings.view(-1, 1, IMAGE_SIZE, IMAGE_SIZE)
        drawings /= 255.0

        # forward
        output = net(drawings)
        loss = F.cross_entropy(output, labels)

        # accuracy
        pred = output.data.max(1)[1]
        correct += float(pred.eq(labels.data).sum())

        # test loss average
        loss_avg += float(loss)

    print(f'Test loss: {loss_avg/len(test_loader)}')
    
    global test_accuracy
    test_accuracy = correct/len(test_loader.dataset)
    
for epoch in range(MODEL_CFG['epochs']):
    print("\nEPOCH: "+str(epoch+1))
    
    if epoch+1 in MODEL_CFG['lr_decay_step']:
        MODEL_CFG['learning_rate'] *= MODEL_CFG['gamma']
        for param_group in optimizer.param_groups:
            param_group['lr'] = MODEL_CFG['learning_rate']
            
    train()
    test()
    
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(net.state_dict(), os.path.join(MODELS_DIR, 'model.pth'))
        print("\nnew best accuracy: %.4f" % best_accuracy)

Train images: 40000
Test images: 10000
Training:   0%|          | 1/313 [00:00<00:51,  6.08it/s]
EPOCH: 1
Training:   4%|▍         | 13/313 [00:00<00:02, 126.81it/s]Test loss: 1.3090746100944808

new best accuracy: 0.5392

EPOCH: 2
Training:   6%|▋         | 20/313 [00:00<00:01, 196.57it/s]Test loss: 0.5216582715511322

new best accuracy: 0.8449

EPOCH: 3
Training:   7%|▋         | 22/313 [00:00<00:01, 212.83it/s]Test loss: 0.42946801796744144

new best accuracy: 0.8695

EPOCH: 4
Training:   6%|▋         | 20/313 [00:00<00:01, 194.97it/s]Test loss: 0.37924542853349374

new best accuracy: 0.8907

EPOCH: 5
Training:   6%|▋         | 20/313 [00:00<00:01, 197.55it/s]Test loss: 0.3482901155948639

new best accuracy: 0.8972

EPOCH: 6
Training:   6%|▋         | 20/313 [00:00<00:01, 199.99it/s]Test loss: 0.329087269645703

new best accuracy: 0.9027

EPOCH: 7
Training:   6%|▋         | 20/313 [00:00<00:01, 197.11it/s]Test loss: 0.3175371760808969

new best accuracy: 0.9077

EPOCH: 8
Training:  